In [ ]:
from paus_utils import *
from jpasLAEs.utils import flux_to_mag

from LAE_selection_method import select_LAEs

import numpy as np

import matplotlib
import matplotlib.pyplot as plt
matplotlib.rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
matplotlib.rc('text', usetex=True)
matplotlib.rcParams.update({'font.size': 16})


In [ ]:
# Load QSO mock
from load_paus_mocks import add_errors, load_qso_mock

source_cats_dir = '/home/alberto/almacen/Source_cats'
mock_path = f'{source_cats_dir}/QSO_PAUS_LAES_2'
mock_QSO_cont_path = f'{source_cats_dir}/QSO_PAUS_contaminants_2'
cat = load_qso_mock(mock_path, mock_QSO_cont_path)

field_name = 'W3'
cat['flx'], cat['err'] = add_errors(cat['flx_0'], field_name, True)

# Compute r_mag
cat['r_mag'] = flux_to_mag(cat['flx'][-4], w_central[-4])

N_sources = len(cat['r_mag'])
print(N_sources)

In [ ]:
# Select LAEs
nb_min, nb_max = 0, 2
r_min, r_max = 17, 24

print(f'Searching for LAEs at {z_NB(nb_min):0.2f} < z < {z_NB(nb_max):0.2f}')

cat = select_LAEs(cat, nb_min, nb_max, r_min, r_max,
                  check_nice_z=True)

In [ ]:
from LAE_selection_method import estimate_continuum

# Estimate the continuum (again)
cont_est, cont_err = estimate_continuum(cat['flx'], cat['err'],
                                        IGM_T_correct=False, N_nb=3)

In [ ]:
# Let's assume the selected line is CIV, and compute the proxy CIV redshift
# The we compute the CIII flux in the appropriate position
w_CIV = 1549.48
w_CIII = 1908.734
w_SiOIV = 1399.8

z_CIV_proxy = w_central[cat['lya_NB']] / w_CIV - 1

nb_CIII_proxy = np.ones(N_sources).astype(int)
for src in range(N_sources):
    nb_CIII_proxy[src] = NB_z(z_CIV_proxy[src], w_CIII)

In [ ]:
# Compute the CIII & CIV flux
CIV_flx = (cat['flx'][cat['lya_NB'], np.arange(N_sources)]
            - cont_est[cat['lya_NB'], np.arange(N_sources)])
CIII_flx = (cat['flx'][nb_CIII_proxy, np.arange(N_sources)]
            - cont_est[nb_CIII_proxy, np.arange(N_sources)])

CIV_err = (cat['err'][cat['lya_NB'], np.arange(N_sources)] ** 2
           + cont_err[cat['lya_NB'], np.arange(N_sources)] ** 2) ** 0.5
CIII_err = (cat['err'][nb_CIII_proxy, np.arange(N_sources)] ** 2
            - cont_err[nb_CIII_proxy, np.arange(N_sources)] ** 2) ** 0.5

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))

mask = ~cat['nice_z'] & cat['nice_lya']
ax.scatter(CIV_flx[mask], CIII_flx[mask],
           c='r', s=2)

mask = cat['nice_z'] & cat['nice_lya']
ax.scatter(CIV_flx[mask], CIII_flx[mask],
           c='g', s=2)

ax.set_ylim(1e-22, 1e-15)
ax.set_xlim(1e-18, 1e-15)

ax.set_xlabel('CIV flux [erg\,s${-1}$]')
ax.set_ylabel('CIII flux [erg\,s${-1}$]')

ax.set_yscale('log')
ax.set_xscale('log')

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))

mask = ~cat['nice_z'] & cat['nice_lya']
ax.scatter(CIV_flx[mask], CIII_flx[mask] / CIII_err[mask],
           c='r', s=2)

mask = cat['nice_z'] & cat['nice_lya']
ax.scatter(CIV_flx[mask], CIII_flx[mask] / CIII_err[mask],
           c='g', s=2)

# ax.set_ylim(-10, 100)
ax.set_xlim(1e-18, 1e-15)

ax.set_xlabel('CIV flux [erg\,s${-1}$]')
ax.set_ylabel('CIII S/N')

ax.set_xscale('log')
ax.set_yscale('log')

plt.show()

In [ ]:
# selection = np.random.permutation(np.where(cat['nice_lya'] & ~cat['nice_z'])[0])
selection = np.random.permutation(np.arange(N_sources))

for i, src in enumerate(selection):
    if i > 9:
        break

    # print(f'z_NB = {cat["z_NB"][src]:0.2f}')
    # print(f'z_spec = {cat["zspec"][src]:0.2f}')

    fig, ax = plt.subplots(figsize=(9, 4))

    plot_PAUS_source(cat['flx'][:, src], cat['err'][:, src],
                     ax=ax, plot_BBs=True, set_ylim=False)

    # ax.axvline(w_central[cat['lya_NB'][src]], ls='--', c='k')
    # ax.axvline(w_central[nb_CIII_proxy[src]], ls='--', c='k')

    # # Plot the real Lya, CIII and CIV lines
    # ax.axvline(w_lya * (cat['zspec'][src] + 1), ls=':', c='r')
    # ax.axvline(w_CIII * (cat['zspec'][src] + 1), ls=':', c='orange')
    # ax.axvline(w_CIV * (cat['zspec'][src] + 1), ls=':', c='orange')
    # ax.axvline(w_SiOIV * (cat['zspec'][src] + 1), ls=':', c='orange')

    ax.errorbar(w_central[np.arange(40)], 1e17*cont_est[:, src], yerr=cont_err[:, src])

    plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))

to_hist = (w_central[cat['lya_NB']] / (1 + cat['zspec']))[cat['nice_lya'] & ~cat['nice_z']]
ax.hist(to_hist, np.linspace(1000, 3500, 100))

ax.axvline(w_lya, ls='--', c='r')
ax.axvline(w_CIV, ls='--', c='r')
ax.axvline(w_CIII, ls='--', c='r')
ax.axvline(w_SiOIV, ls='--', c='orange')

plt.show()